In [1]:
%%writefile cuda_stuff.cuh
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

#ifndef cuda_stuff_H
#define cuda_stuff_H

//MACRO TO DEBUG CUDA FUNCTIONS
/** Error checking,
 *  taken from https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
 */
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

#endif

Writing cuda_stuff.cuh


## Comparison on size, blocks and threads

In [2]:
%%writefile saxpy.cu
/*
 * GPU code of SAPXPY
 * Y = a.X + Y
 */

#include <stdlib.h>
#include <stdio.h>
#include <cuda.h>
#include <math.h>

#include "cuda_stuff.cuh"

////////////////////////////////////////////////////////////////
//     Vector initialization
////////////////////////////////////////////////////////////////
void init_tab(float *tab, int len, float val) {
    for (int k=0; k<len; k++)
      tab[k]= val;
}

void print_tab(const char *tab_name, float *tab, int len){
   int k;
   printf("\n 10 first elements of %s: \n", tab_name);
   for (k=0; k<10; k++)
      printf("%.2f ",tab[k]);
   printf("\n 10 lasts : \n");
   for (k=len-10; k<len; k++)
      printf("%.2f ",tab[k]);
   printf("\n");
}



////////////////////////////////////////////////////////////////
//     SAXPY kernel
////////////////////////////////////////////////////////////////
__global__ void saxpy(float *tabX, float *tabY, int len, float a){
   // TODO
   int idx = blockIdx.x * blockDim.x + threadIdx.x;

   if(idx < len)
     tabY[idx] = a * tabX[idx] + tabY[idx];
}




////////////////////////////////////////////////////////////////
//     Main program
////////////////////////////////////////////////////////////////
int main( int argc, char** argv){
    float *tabX_d, *tabX_h;
    float *tabY_d, *tabY_h;
    int len_count = 3;
    int lens[len_count] = {1000, 10000, 100000};
    int num_threads[len_count] = {256, 512, 1024};

    for (int i=0; i<len_count; i++){
        int len = lens[i];
        for (int j=0; j<len_count; j++){
            int num_thread = num_threads[j];

            cudaEvent_t start, stop;
            float milliseconds = 0.0;
            gpuErrchk(cudaEventCreate(&start));
            gpuErrchk(cudaEventCreate(&stop));

            /** Initialization of the grid **/
            // TODO
            dim3 dimBlock (num_thread);
            dim3 dimGrid( ceil(len / (float) dimBlock.x) );

            /** Allocation in host memory **/
            tabX_h = (float *) malloc(sizeof(float) * len);
            init_tab(tabX_h, len , 5.);
            //TODO - allocation and initialization of tabY_dh
            tabY_h = (float *) malloc(sizeof(float) * len);
            init_tab(tabY_h, len, 1.);

            /** Allocation in device memory **/
            gpuErrchk(cudaMalloc((void**) &tabX_d, sizeof(float) * len));
            // TODO - allocation of tabY_d
            gpuErrchk(cudaMalloc((void**) &tabY_d, sizeof(float) * len));


            /** Pre-print of tabY **/
            printf("Array length: %d\n", len);
            printf("Number of threads: %d\n", num_thread);
            printf("Number of blocks: %d\n", dimGrid.x);
            //printf("\nBefore computation \n");
            //print_tab("tabY_h", tabY_h, len);


            /** Transfer of data from host to device **/
            // TODO
            gpuErrchk(cudaMemcpy(tabX_d, tabX_h, sizeof(float) * len, cudaMemcpyHostToDevice));
            gpuErrchk(cudaMemcpy(tabY_d, tabY_h, sizeof(float) * len, cudaMemcpyHostToDevice));

            /** SaxPY kernel launching **/
            //TODO
            gpuErrchk(cudaEventRecord(start));
            saxpy<<<dimGrid, dimBlock>>>(tabX_d, tabY_d, len, 2.);
            gpuErrchk( cudaPeekAtLastError() );
            gpuErrchk( cudaDeviceSynchronize() );
            gpuErrchk(cudaEventRecord(stop));
            gpuErrchk(cudaEventSynchronize(stop));
            gpuErrchk(cudaEventElapsedTime(&milliseconds, start, stop));

            /** Transfer of the result from device to host **/
            // TODO
            gpuErrchk(cudaMemcpy(tabY_h, tabY_d, sizeof(float) * len, cudaMemcpyDeviceToHost));

            /** Affichage du resultat **/
            //printf("\nAfter computation\n");
            //print_tab("tabY_h", tabY_h, len);

            /** Memory free **/
            gpuErrchk(cudaFree(tabX_d)); gpuErrchk(cudaFree(tabY_d));
            free(tabX_h); free(tabY_h);

            printf("\nExecution time: %f ms\n", milliseconds);
            printf("\n-----------------------\n\n");
        }
    }

    return EXIT_SUCCESS;
}

Writing saxpy.cu


In [3]:
! nvcc -arch=sm_75 saxpy.cu -o saxpy

In [4]:
! ./saxpy

Array length: 1000
Number of threads: 256
Number of blocks: 4

Execution time: 0.147232 ms

-----------------------

Array length: 1000
Number of threads: 512
Number of blocks: 2

Execution time: 0.020128 ms

-----------------------

Array length: 1000
Number of threads: 1024
Number of blocks: 1

Execution time: 0.018240 ms

-----------------------

Array length: 10000
Number of threads: 256
Number of blocks: 40

Execution time: 0.012384 ms

-----------------------

Array length: 10000
Number of threads: 512
Number of blocks: 20

Execution time: 0.014592 ms

-----------------------

Array length: 10000
Number of threads: 1024
Number of blocks: 10

Execution time: 0.013024 ms

-----------------------

Array length: 100000
Number of threads: 256
Number of blocks: 391

Execution time: 0.017728 ms

-----------------------

Array length: 100000
Number of threads: 512
Number of blocks: 196

Execution time: 0.021120 ms

-----------------------

Array length: 100000
Number of threads: 1024
Num

**Interpretation**

Using 1024 threads per block is optimal for every lenght of the array. The difference between threads per block is decreased as the length of the array decreases.

## Comparison CPU vs GPU

In [5]:
%%writefile saxpy.cu
/*
 * GPU code of SAPXPY
 * Y = a.X + Y
 */

#include <stdlib.h>
#include <stdio.h>
#include <cuda.h>
#include <math.h>
#include <sys/time.h>

#include "cuda_stuff.cuh"

////////////////////////////////////////////////////////////////
//     Vector initialization
////////////////////////////////////////////////////////////////
void init_tab(float *tab, int len, float val) {
    for (int k=0; k<len; k++)
      tab[k]= val;
}

void print_tab(const char *tab_name, float *tab, int len){
   int k;
   printf("\n 10 first elements of %s: \n", tab_name);
   for (k=0; k<10; k++)
      printf("%.2f ",tab[k]);
   printf("\n 10 lasts : \n");
   for (k=len-10; k<len; k++)
      printf("%.2f ",tab[k]);
   printf("\n");
}


////////////////////////////////////////////////////////////////
//     CPU SAXPY implementation
////////////////////////////////////////////////////////////////
void saxpy_cpu(float *tabX, float *tabY, int len, float a) {
    for (int i = 0; i < len; i++) {
        tabY[i] = a * tabX[i] + tabY[i];
    }
}


////////////////////////////////////////////////////////////////
//     SAXPY kernel
////////////////////////////////////////////////////////////////
__global__ void saxpy(float *tabX, float *tabY, int len, float a){
   // TODO
   int idx = blockIdx.x * blockDim.x + threadIdx.x;

   if(idx < len)
     tabY[idx] = a * tabX[idx] + tabY[idx];
}


////////////////////////////////////////////////////////////////
//     Get current time in milliseconds
////////////////////////////////////////////////////////////////
double get_time_ms() {
    struct timeval tv;
    gettimeofday(&tv, NULL);
    return tv.tv_sec * 1000.0 + tv.tv_usec / 1000.0;
}


////////////////////////////////////////////////////////////////
//     Main program
////////////////////////////////////////////////////////////////
int main( int argc, char** argv){
    float *tabX_d, *tabX_h, *tabX_cpu;
    float *tabY_d, *tabY_h, *tabY_cpu;
    int len_count = 3;
    int lens[len_count] = {1000, 10000, 100000};

    for (int i=0; i<len_count; i++){
        int len = lens[i];

        /** CPU computation **/
        tabX_cpu = (float *) malloc(sizeof(float) * len);
        init_tab(tabX_cpu, len, 5.);
        tabY_cpu = (float *) malloc(sizeof(float) * len);
        init_tab(tabY_cpu, len, 1.);

        double cpu_start = get_time_ms();
        saxpy_cpu(tabX_cpu, tabY_cpu, len, 2.);
        double cpu_stop = get_time_ms();
        float cpu_milliseconds = cpu_stop - cpu_start;

        cudaEvent_t gpu_start, gpu_stop;
        float gpu_milliseconds = 0.0;
        gpuErrchk(cudaEventCreate(&gpu_start));
        gpuErrchk(cudaEventCreate(&gpu_stop));

        /** Initialization of the grid **/
        // TODO
        dim3 dimBlock (1024);
        dim3 dimGrid( ceil(len / (float) dimBlock.x) );

        /** Allocation in host memory **/
        tabX_h = (float *) malloc(sizeof(float) * len);
        init_tab(tabX_h, len , 5.);
        //TODO - allocation and initialization of tabY_dh
        tabY_h = (float *) malloc(sizeof(float) * len);
        init_tab(tabY_h, len, 1.);

        /** Allocation in device memory **/
        gpuErrchk(cudaMalloc((void**) &tabX_d, sizeof(float) * len));
        // TODO - allocation of tabY_d
        gpuErrchk(cudaMalloc((void**) &tabY_d, sizeof(float) * len));


        /** Pre-print of tabY **/
        printf("Array length: %d\n", len);
        printf("Number of threads: %d\n", dimBlock.x);
        printf("Number of blocks: %d\n", dimGrid.x);
        //printf("\nBefore computation \n");
        //print_tab("tabY_h", tabY_h, len);


        /** Transfer of data from host to device **/
        // TODO
        gpuErrchk(cudaMemcpy(tabX_d, tabX_h, sizeof(float) * len, cudaMemcpyHostToDevice));
        gpuErrchk(cudaMemcpy(tabY_d, tabY_h, sizeof(float) * len, cudaMemcpyHostToDevice));

        /** SaxPY kernel launching **/
        //TODO
        gpuErrchk(cudaEventRecord(gpu_start));
        saxpy<<<dimGrid, dimBlock>>>(tabX_d, tabY_d, len, 2.);
        gpuErrchk( cudaPeekAtLastError() );
        gpuErrchk( cudaDeviceSynchronize() );
        gpuErrchk(cudaEventRecord(gpu_stop));
        gpuErrchk(cudaEventSynchronize(gpu_stop));
        gpuErrchk(cudaEventElapsedTime(&gpu_milliseconds, gpu_start, gpu_stop));

        /** Transfer of the result from device to host **/
        // TODO
        gpuErrchk(cudaMemcpy(tabY_h, tabY_d, sizeof(float) * len, cudaMemcpyDeviceToHost));

        /** Affichage du resultat **/
        //printf("\nAfter computation\n");
        //print_tab("tabY_h", tabY_h, len);

        /** Memory free **/
        gpuErrchk(cudaFree(tabX_d)); gpuErrchk(cudaFree(tabY_d));
        free(tabX_h); free(tabY_h);

        printf("\nExecution time:\n");
        printf("CPU: %f ms, GPU: %f ms", cpu_milliseconds, gpu_milliseconds);
        printf("\n-----------------------\n\n");
    }

    return EXIT_SUCCESS;
}

Overwriting saxpy.cu


In [6]:
! nvcc -arch=sm_75 saxpy.cu -o saxpy

In [7]:
! ./saxpy

Array length: 1000
Number of threads: 1024
Number of blocks: 1

Execution time:
CPU: 0.003906 ms, GPU: 0.149152 ms
-----------------------

Array length: 10000
Number of threads: 1024
Number of blocks: 10

Execution time:
CPU: 0.038086 ms, GPU: 0.017696 ms
-----------------------

Array length: 100000
Number of threads: 1024
Number of blocks: 98

Execution time:
CPU: 0.307861 ms, GPU: 0.023296 ms
-----------------------



**Interpretation**

The execution time with CPU is proportional to the length of the array. On the other hand, the GPU execution times stay constant (regardless of length increase).